In [1]:
import pandas as pd
import json
from itertools import product

from utils import process_authors, add_citations_to_authors, process_n_citations
from visualization import visualize_barchart

%load_ext autoreload
%autoreload 2

In [2]:
data = pd.read_csv('../clustering/results/004_of_V13_topics.csv')

In [3]:
data.drop(data[data['authors'].map(lambda x: '\\' in r'%r' % x)].index, inplace=True)
data['authors'] = data['authors'].apply(lambda x: json.loads(x.replace('\'', '\"')))
data['authors'] = data['authors'].apply(lambda x:  process_authors(x))
data.drop(data[data['authors'].map(lambda x: x == '[]')].index, inplace=True)
data['authors'] = data.apply(lambda row: add_citations_to_authors(row), axis=1)

### Most citated authors in each topic

In [4]:
data_grouped = data.groupby(by='topic')['authors'].sum().reset_index(name ='authors')
data_grouped['authors'] = data_grouped['authors'].apply(lambda x: process_n_citations(x))

In [5]:
data_grouped.to_csv('data/grouped_by_topic_with_author_num_citation.csv')

In [6]:
fig = visualize_barchart(data_grouped.to_dict(orient='records'))
# fig.show()

In [7]:
fig.write_html('html/top_citation_authors.html')

### Network Graph

In [14]:
from pyvis.network import Network
net = Network(notebook=True)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [19]:
import itertools


unique_authors = set()
edges = {}
for i in range(len(data[:500])):
    author_names = set()
    for author in data.iloc[i]['authors']:
        if 'name' in author:
            author_names.add(author['name'])
    edges[data.iloc[i]['title']] = list(itertools.product(list(author_names), list(author_names)))
    unique_authors = unique_authors.union(author_names)

In [20]:
net.add_nodes(list(unique_authors))

In [21]:
for article in edges:
    for pair in edges[article]:
        if pair[0] != pair[1]:
            net.add_edge(pair[0], pair[1])

In [22]:
net.show('html/nework_graph.html')